# Gray-Scott Model of Reaction Diffusion

The Gray-Scott model of a reaction diffusion system with two interacting species $U$ and $V$ is given by:
$$
\begin{align}
\frac{\partial{u}}{\partial{t}} &= D_u\nabla^2u - uv^2 + f(1-u) \\
\frac{\partial{v}}{\partial{t}} &= D_v\nabla^2v + uv^2 - (f+k)v
\end{align}
$$

where $u$ and $v$ represent the concentrations of the two species $U$ and $V$, respectively, $D_u$ and $D_v$ their respective diffusion constants, and $f$ and $k$ the feed rate and kill rate, respectively. $\nabla^2$ is the Laplace operator, which in this two-dimensional problem is given by:

$$ \nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}.$$

For different parameter choices, this model can producing very diverse patterns which mimic patterns found in nature, such as spots and stripes. At the microscopic level, two chemical reactions govern the system above:

$$
\begin{align}
U + 2V &\rightarrow 3V \\
V &\rightarrow P
\end{align}
$$

where $P$ represents an inert product which depletes the amount of $V$ present. Our goal will be to explore the variety of interesting visual patterns this simple yet rich interacting system can exhibit. For reference, the following website offers an interactive look into the phase diagram spanned by the $f$ and $k$ parameters: [Reaction-Diffusion by the Gray-Scott Model: Pearson's Parametrization](http://mrob.com/pub/comp/xmorphia/index.html).


We'll start by importing the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import ipywidgets as widgets
import gc
from skimage.data import binary_blobs, checkerboard
from skimage.io import imread
from scipy.signal import convolve2d
from scipy.integrate import solve_ivp
from reaction_diffusion_utils import *
from IPython.display import Image, display, update_display, clear_output

The initialization of our system requires preparing two $N\times N$ grids: one for the concentration $u$ of the first species, and the other the concentration $v$ of the second species. We will use a built-in function ```binary_blobs``` which generates a binary image of random, blob-like structures. Where our image is 1, we will initialize $u$ to $0.5$, and to $1$ otherwise. Where our image is 1, we will initialize $v$ to $0.25$, and to $0$ otherwise.

In [ ]:
def init_blobs(N, seed=12):
    '''Initialize values of a two-component system randomly.'''
    np.random.seed(seed)
    mask = binary_blobs(N, blob_size_fraction=0.1, volume_fraction=0.3, seed=seed)
    u = np.ones((N,N)); u[mask] = 0.5
    v = np.zeros((N,N)); v[mask] = 0.25
    return u, v

Let's plot an example of the initial state. Note that you can change the color palette used for plotting by changing the ```colormap``` argument in ```plot_pattern``` below to the name of one of these [colormaps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html).

In [ ]:
# initialize
N = 100
u, v = init_blobs(N, seed=12)

# plot the u-component
fig, ax, im = plot_pattern(u, colormap='twilight');

We can discretize the Laplace operator $\nabla^2$ using a 2D convolution operation with the kernel:

$$
\begin{bmatrix}
0.05 & 0.2 & 0.05 \\
0.2 & -1 & 0.2 \\
0.05 & 0.2 & 0.05
\end{bmatrix}
$$

You'll notice that the sum over all values of the discrete Laplacian is zero. This enforces mass conservation; since the Laplacian of the concentration is proportional to the rate of inflow or outflow from any point in space, the outflow from any given point must equal the sum of inflow contributions that point makes to any of its neighbors. 

Since we'll need to perform two Laplace operations at each integration step, one for each species concentration, we will write a general function for the Laplace operator that can be repurposed several times.

In [ ]:
# use convolution to compute the Laplacian.
def laplacian2D(u, N):
    '''Compute the Laplacian of concentration.
    u - a linear array of concentrations (linear for compatibility with solver).
    N - the system size in each dimension (2D).
    '''    
    kernel = np.array([[0.05,0.2,0.05],
                       [0.2,-1,0.2],
                       [0.05,0.2,0.05]])

    lap = convolve2d(u.reshape((N,N)), kernel,
                     mode = 'same',
                     boundary = 'wrap') # periodic boundaries
    return lap.ravel()

In [ ]:
Lu = laplacian2D(u, N)
print(Lu)

Our original set of equations is a system of PDEs, but by discretizing the spatial dimensions on an $N\times N$ grid, we convert our system to a set of ODEs with time as the continuous integration variable. The Gray-Scott system of ODEs is defined in the function ```gray_scott``` below.

In [ ]:
def gray_scott(t, q, N, Du, Dv, f, k):
    '''Spatially discretized PDE for the Gray-Scott reaction-diffusion system.
        t - the current time.
        q - a linear array of concentrations u and v.
        N - the system size in each dimension.
        Du, Dv - the diffusion constants.
        f - the feed rate.
        k - the kill rate.
    '''
    u, v = q[:N*N], q[N*N:] # unpack the species concentrations
    dq = np.zeros(2*N*N)
    
    # compute the Laplacians.
    Lu = laplacian2D(u, N)
    Lv = laplacian2D(v, N)

    du = Du*Lu - u*v*v + f*(1 - u)
    dv = Dv*Lv + u*v*v - (f + k)*v
    
    dq[:N*N], dq[N*N:] = du, dv
    return dq

In [ ]:
# parameters
Du = 0.3
Dv = 0.1
f = 0.06
k = 0.063

t = 0
q = np.hstack([u.ravel(), v.ravel()])

dq = gray_scott(t, q, N, Du, Dv, f, k)
print(dq)

Finally, the function ```solve``` integrates the ODEs using the function ```solve_ivp``` from the Python ```scipy``` library.

In [ ]:
def solve(fun, ti, tf, nt, qi, args, rtol=1e-6, atol=1e-6):
    '''
    fun - the function to integrate
    ti, tf - starting and ending integration times.
    nt - number of equally spaced output points on [ti, tf].
    qi - initial conditions [u, v].
    args - tuple of model parameters (Du, Dv, a, b, c).
    rtol, atol - tolerance for tuning the accuracy of the solver.
    '''
    u, v = qi                           # unpack initial components
    N = len(u)                          # extract grid dimension
    t_eval = np.linspace(ti, tf, nt)    # evaluation points
    Du, Dv, f, k = args                 # unpack extra arguments
    
    start_time = time.time()            # time the integration
    
    # integrate the differential equation using finite differences
    sol = solve_ivp(lambda t, y: fun(t, y, N, Du, Dv, f, k), [ti, tf], np.hstack([u.ravel(), v.ravel()]),
                    t_eval=t_eval, rtol=rtol, atol=atol, method='RK23')
    u = sol.y[:N*N,:].reshape(N,N,len(t_eval))
    
    # print elapsed time
    print('elapsed time (s): ', time.time() - start_time)
    
    return u

In [ ]:
ti = 0
tf = 3000
frames = 100

# solve
out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))

In [ ]:
# create animation
ani = animate_pattern(out, colormap='twilight')

# save animation as a gif and display the movie
ani.save('example.gif', writer='pillow', fps=20)
Image('example.gif')

With our integration routine set up, let's explore some of the varied patterns of the Gray-Scott system using an interactive widget. Use the slider to explore the resulting phases when varying the ratio of diffusion constants $D_u/D_v$, and the feed and kill rates (some presets and the descriptions of their phases are given as guidance).

In [ ]:
diffusion_slider = widgets.FloatSlider(
    value=3., min=2., max=4., step=0.1,
    description='Du/Dv:', readout_format='.1f')

@widgets.interact_manual(d=diffusion_slider,
                         presets = ['1. f=0.06, k=0.063',
                                    '2. f=0.02, k=0.048',
                                    '3. f=0.041, k=0.057',
                                    '4. f=0.02, k=0.057'])
def interactive_menu_plot(d=4, presets='1. f=0.06, k=0.063'):
    h = display(display_id=True)
    
    params = [[0.06, 0.063], # blocky maze
              [0.02, 0.048], # rounded maze
              [0.041, 0.057],# bubbles
              [0.02, 0.057]] # cell division

    
    # get parameters
    f, k = params[eval(presets[0])-1]
    Du = 0.3
    Dv = Du/d
    
    # initialize u and v
    N = 100
    u, v = init_blobs(N, seed=12)
    
    # solve
    ti = 0
    tf = int(10000/d)
    frames = 100
    out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))

    # create animation
    print('creating animation...')
    ani = animate_pattern(out, colormap='twilight')

    # save animation as a gif and display the movie
    ani.save('example.gif', writer='pillow', fps=20)
    h.display(Image('example.gif'))
    gc.collect()

To more flexibly vary the model parameters, the widget below provides sliders for the feed and kill rates. You can use the above presets as starting points for finding your own patterns!

In [ ]:
diffusion_slider = widgets.FloatSlider(
    value=3., min=2., max=4., step=0.1,
    description='Du/Dv:', readout_format='.1f')
feed_slider = widgets.FloatSlider(
    value=0.03, min=0.005, max=0.06, step=0.001,
    description='Feed rate:', readout_format='.3f',)
kill_slider = widgets.FloatSlider(
    value=0.065, min=0.015, max=0.075, step=0.001,
    description='Kill rate:', readout_format='.3f',)

@widgets.interact_manual(
    d=diffusion_slider, f=feed_slider, k=kill_slider)
def interactive_slider_plot(d=3., f=0.045, k=0.063):
    h = display(display_id=True)
    
    Du = 0.3
    Dv = Du/d
    
    # initialize u and v
    N = 100
    u, v = init_blobs(N, seed=12)
    
    # solve
    ti = 0
    tf = int(10000/d)
    frames = 100
    out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))
    
    # create animation
    print('creating animation...')
    ani = animate_pattern(out, colormap='twilight')

    # save animation as a gif and display the movie
    ani.save('example.gif', writer='pillow', fps=20)
    h.display(Image('example.gif'))
    gc.collect()

Use the following two cells to manually vary any of the parameters, and then save and display an animation of the result.

In [ ]:
# set parameters
Du = 0.3
Dv = 0.1
f = 0.06
k = 0.063

# initialize u and v
N = 100
u, v = init_blobs(N, seed=12)
plot_pattern(u, colormap='twilight')
plt.show()

# solve
ti = 0
tf = 3000
frames = 100
out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))

In [ ]:
# create animation
ani = animate_pattern(out, colormap='twilight')

# save animation as a gif and display the movie
ani.save('example.gif', writer='pillow', fps=20)
Image('example.gif')

Let's try seeding the initial image with a different mask.

In [ ]:
def init_checkerboard(N):
    '''Initialize values of a two-component system on a checkerboard pattern.'''
    mask = checkerboard()
    
    # bin mask
    s = [int(np.floor(i/N)) for i in mask.shape]
    if min(s)>1:
        mask = mask[::s[0],::s[1]]
    
    # truncate and binarize mask
    mask = mask[:N,:N]
    mask[mask > 0] = 1
    
    u = np.ones((N,N)); u[mask.astype(np.bool)] = 0.
    v = np.zeros((N,N)); v[mask.astype(np.bool)] = 1.
    return u, v

In [ ]:
# set parameters
Du = 0.3
Dv = 0.1
f = 0.06
k = 0.063

# initialize u and v
N = 100
u, v = init_checkerboard(N)
plot_pattern(u, colormap='twilight');
plt.show()

# solve
ti = 0
tf = 3000
frames = 100
out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))

In [ ]:
# create animation
ani = animate_pattern(out, colormap='twilight')

# save animation as a gif and display the movie
ani.save('example.gif', writer='pillow', fps=20)
Image('example.gif')

Input a link to a custom initial image.

In [ ]:
def init_custom(N):
    '''Initialize values of a two-component system on a custom pattern.'''
    mask = imread("https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Five_Pointed_Star_Solid.svg/1087px-Five_Pointed_Star_Solid.svg.png", as_gray=True)
    
    # bin mask
    s = [int(np.floor(i/N)) for i in mask.shape]
    if min(s):
        mask = mask[::s[0],::s[1]]
    
    # truncate and binarize mask
    mask = mask[:N,:N]
    mask[mask > 0] = 1

    # initialize concentrations
    u = np.ones((N,N)); u[mask.astype(np.bool)] = 0.
    v = np.zeros((N,N)); v[mask.astype(np.bool)] = 1.
    return u, v

In [ ]:
# set parameters
Du = 0.3
Dv = 0.1
f = 0.06
k = 0.063

# initialize u and v
N = 100
u, v = init_custom(N)
plot_pattern(u, colormap='twilight');
plt.show()

# solve
ti = 0
tf = 3000
frames = 100
out = solve(gray_scott, ti, tf, frames, [u, v], args = (Du, Dv, f, k))

In [ ]:
# create animation
ani = animate_pattern(out, colormap='twilight')

# save animation as a gif and display the movie
ani.save('example.gif', writer='pillow', fps=20)
Image('example.gif')